In [1]:
from mtcnn import MTCNN
from PIL import Image
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Face Images.csv").sample(frac=0.01)

In [3]:
def crop_image(image, box):
    # Input: image: np array of pixels (image) to crop
    #        box: a dictionary returned by detect_faces() containing coordinates/width/height of the box outlining the detected face
    # Output: an np array of pixels representing the cropped image
    x, y, width, height = box[0], box[1], box[2], box[3] # x,y,w,h of the box containing a detected face
    return image[y:y+height, x:x+width]

In [4]:
def resize_image(image):
    # Input: image: an np array of pixels
    # Output: an np array representing the input image resized to 224x224 px
    size = (224, 224)
    return np.array(Image.fromarray(np.squeeze(image, axis=2)).resize(size))

In [ ]:
# For each image in the df, use the mtcnn to detect the face, then crop the image to just the face portion, and then resize to a constant size of 224x224 px

images = []
detector = MTCNN()

for index, row in df.iterrows():
    original_image = np.fromstring(row["pixels"], dtype=int, sep=" ").reshape(row["height"], row["width"], 1).astype("float32")
    img = np.tile(original_image, [1,1,3]) # this MTCNN library only works with colour images (imgs with 3 channels), but grayscale images have only 1 channel. This pads the img to make the image have 3 channels
    output = detector.detect_faces(img)
    if len(output) == 1: # 1 face detected
        resized_image = resize_image(crop_image(original_image, output[0]["box"]))
        images.append([row["emotion"], resized_image])